In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gp
import os
from tqdm import tqdm
import folium
import warnings
warnings.filterwarnings("ignore")

%pwd

'/home/cseadmin/data/cys/TrafficFlowModel/data_process'

In [2]:
# 获取taxi中所有文件，之后读取使用
root_path = os.walk("../data/taxi")
all_files = []
for root,ds,fs in root_path:
    days = []
    for f in fs:
        full_path = os.path.join(root,f)
        days.append(full_path)
    if len(days)==0:
        continue
    all_files.append(days)
all_files

[['../data/taxi/2019-12-10/2019-12-10_7',
  '../data/taxi/2019-12-10/2019-12-10_4',
  '../data/taxi/2019-12-10/2019-12-10_5',
  '../data/taxi/2019-12-10/2019-12-10_8',
  '../data/taxi/2019-12-10/2019-12-10_6',
  '../data/taxi/2019-12-10/2019-12-10_1',
  '../data/taxi/2019-12-10/2019-12-10_0',
  '../data/taxi/2019-12-10/2019-12-10_3',
  '../data/taxi/2019-12-10/2019-12-10_2'],
 ['../data/taxi/2019-12-13/2019-12-13_1',
  '../data/taxi/2019-12-13/2019-12-13_0'],
 ['../data/taxi/2019-12-07/2019-12-07_5',
  '../data/taxi/2019-12-07/2019-12-07_0',
  '../data/taxi/2019-12-07/2019-12-07_8',
  '../data/taxi/2019-12-07/2019-12-07_1',
  '../data/taxi/2019-12-07/2019-12-07_6',
  '../data/taxi/2019-12-07/2019-12-07_4',
  '../data/taxi/2019-12-07/2019-12-07_3',
  '../data/taxi/2019-12-07/2019-12-07_2',
  '../data/taxi/2019-12-07/2019-12-07_7'],
 ['../data/taxi/2019-12-08/2019-12-08_8',
  '../data/taxi/2019-12-08/2019-12-08_2',
  '../data/taxi/2019-12-08/2019-12-08_6',
  '../data/taxi/2019-12-08/2019

In [3]:
names=["sys_time", "license_number", "lng", "lat", "gps_time", "EMPTY1", "speed", "direction", "car_status","alarm_status",
       "EMPTY2", "EMPTY3", "license_color", "recorder_speed", "mileage", "height", "EMPTY4"]

# 这两个日期是需要处理的文件日期开头以及结尾
open_day = '2019-12-01'
close_day = '2019-12-14'

# 将数据进行初步处理
# dest_path是中间结果，暂存到一个文件夹中，之后读取再进行处理
dest_path = "../data/taxi_after_proc/proc_segment/"

all_files.sort()
for each_day in tqdm(all_files):
    each_day.sort()
    
    for each_time in tqdm(each_day):
        records = pd.read_csv(each_time,names=names)
        group = records.groupby("license_number")
        for each_lincense, each_record in group:
            posi = each_record[['lat', 'lng', 'gps_time', 'speed', 'direction', 'car_status']]
            # 按gps时间排序，并去重
            posi['gps_time'] = pd.to_datetime(posi['gps_time'])
            posi.sort_values(by="gps_time", inplace=True, ascending=True)
            posi = posi.drop_duplicates()
            # 去除错误日期的记录
            posi = posi[(posi['gps_time'] > open_day) & (posi['gps_time'] < close_day)]
            # 去除速度为0的记录
            posi = posi.loc[(posi != 0).all(axis=1)]
            # 去除数量过少的记录
            if len(posi) < 30:
                continue
            # 将结果输出到dest_path中
            posi.to_csv(dest_path+each_time[-7:]+each_lincense+".csv",index=False)
            

100%|██████████| 13/13 [4:41:37<00:00, 1299.83s/it]


In [ ]:
files=os.listdir(dest_path)
car_set=set()

for file in files:
    car=file[7:-4]
    car_set.add(car)

days=["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

for car in car_set:
    for day in days:
        df_all=pd.DataFrame()
        for i in range(10):
            try:
                df_all=df_all.append(pd.read_csv("../data/taxi_after_proc/proc_segment/12-{}_{}{}.csv".format(day, i, car)))
                df_all = df_all.drop_duplicates(subset="gps_time",keep="first")
                df_all = df_all[(df_all['gps_time']>'2019-12-01')&(df_all['gps_time']<'2019-12-14')]
                df_all = df_all[(df_all['speed']>0)&(df_all['speed']<150)]
#               print(df_all)
            except FileNotFoundError:
                continue
        df_all.to_csv("../data/taxi_after_proc/merged/12-{}_{}.csv".format(day, car))

for file in os.listdir("../data/taxi_after_proc/merged/"):
    df=pd.read_csv("../data/taxi_after_proc/merged/"+file)
    try:
        df=df.drop(columns="Unnamed: 0")
    except KeyError:
        continue
    df.to_csv("../data/taxi_after_proc/merged/"+file, index=False)